#### Random forest model
x = twitter sentiment(positive, negative, neutral), stock volume
  
y = stock price (increase, decrease, no change) (original label)
  
model accuracy = 53%

In [2]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report
import pandas as pd

In [3]:
df2 = pd.read_csv('../Data/combind_sen_stock.csv')
df2 = df2.replace({'positive':1,'negative':-1,'neutral':0,'Increase':1,'Decrease':-1,'No Change':0})
df2.head()

,Unnamed: 0,Date,Open,High,Low,Close,Adj Close,Volume,Label,Tweets_clean_more,polarity,subjectivity,sentiment,half_H_L
0,0,2011-12-01,2.171333,2.266000,2.132000,2.173333,2.173333,15453000,1,i made volum model s go now i need work miniat...,0.300000,0.200000,1,1.200000
1,1,2011-12-21,1.860667,1.871333,1.735333,1.838000,1.838000,25582500,-1,yeah realli mom attest not sure i handl char m...,0.377273,0.708687,1,1.003667
2,2,2011-12-22,1.840000,1.870000,1.820000,1.851333,1.851333,15141000,1,model s option perform red black i deliv car j...,0.090476,0.376190,1,0.960000
3,3,2011-12-28,1.932667,1.949333,1.869333,1.900667,1.900667,8628000,-1,if ever want know real truth moon land best on...,0.600000,0.300000,1,1.014666
4,4,2011-12-29,1.906000,1.956000,1.903333,1.915333,1.915333,7323000,1,cool person essay it realli reson i felt way c...,0.132857,0.438571,1,1.004333


In [4]:
x = df2[['sentiment','Volume','polarity','Adj Close','subjectivity']]
y = df2['Label']
x_train,x_test,y_train,y_test = train_test_split(x,y,train_size=0.8,random_state=123)
y_train = y_train.values.reshape(-1,1)
y_test = y_test.values.reshape(-1,1)

In [5]:
clf = RandomForestClassifier(n_estimators = 100)
clf.fit(x_train,y_train.ravel())
y_pred = clf.predict(x_test)
print("Accuracy Score: ")
print(accuracy_score(y_test,y_pred))
print("--------------------------------------------------------")
print("Report: ")
print(classification_report(y_test,y_pred))


Accuracy Score: 
0.49122807017543857
--------------------------------------------------------
Report: 
              precision    recall  f1-score   support

          -1       0.45      0.53      0.49       128
           0       0.00      0.00      0.00         9
           1       0.54      0.49      0.51       148

    accuracy                           0.49       285
   macro avg       0.33      0.34      0.33       285
weighted avg       0.48      0.49      0.48       285



In [6]:
cm = confusion_matrix(y_test,y_pred)
cm_df = pd.DataFrame(cm, index=["Actual -1", "Actual 0","Actual 1"],columns=["Predicted -1", "Predicted 0","Predicted 1"])
print("Confusion Matrix: ")
display(cm_df)

Confusion Matrix: 


,Predicted -1,Predicted 0,Predicted 1
Actual -1,68,1,59
Actual 0,6,0,3
Actual 1,76,0,72


In [7]:
x = df2[['sentiment','Volume','polarity','Adj Close']]
y = df2['Label']
x_train,x_test,y_train,y_test = train_test_split(x,y,train_size=0.8,random_state=123)
y_train = y_train.values.reshape(-1,1)
y_test = y_test.values.reshape(-1,1)

clf = RandomForestClassifier(n_estimators = 100)
clf.fit(x_train,y_train.ravel())
y_pred = clf.predict(x_test)
print("Accuracy Score: ")
print(accuracy_score(y_test,y_pred))
print("--------------------------------------------------------")
print("Report: ")
print(classification_report(y_test,y_pred))

cm = confusion_matrix(y_test,y_pred)
cm_df = pd.DataFrame(cm, index=["Actual -1", "Actual 0","Actual 1"],columns=["Predicted -1", "Predicted 0","Predicted 1"])
print("Confusion Matrix: ")
display(cm_df)

Accuracy Score: 
0.4631578947368421
--------------------------------------------------------
Report: 
              precision    recall  f1-score   support

          -1       0.42      0.47      0.44       128
           0       0.00      0.00      0.00         9
           1       0.52      0.49      0.50       148

    accuracy                           0.46       285
   macro avg       0.31      0.32      0.31       285
weighted avg       0.46      0.46      0.46       285

Confusion Matrix: 


,Predicted -1,Predicted 0,Predicted 1
Actual -1,60,3,65
Actual 0,7,0,2
Actual 1,76,0,72


In [8]:
x = df2[['sentiment','Volume','polarity']]
y = df2['Label']
x_train,x_test,y_train,y_test = train_test_split(x,y,train_size=0.8,random_state=123)
y_train = y_train.values.reshape(-1,1)
y_test = y_test.values.reshape(-1,1)

clf = RandomForestClassifier(n_estimators = 100)
clf.fit(x_train,y_train.ravel())
y_pred = clf.predict(x_test)
print("Accuracy Score: ")
print(accuracy_score(y_test,y_pred))
print("--------------------------------------------------------")
print("Report: ")
print(classification_report(y_test,y_pred))

cm = confusion_matrix(y_test,y_pred)
cm_df = pd.DataFrame(cm, index=["Actual -1", "Actual 0","Actual 1"],columns=["Predicted -1", "Predicted 0","Predicted 1"])
print("Confusion Matrix: ")
display(cm_df)

Accuracy Score: 
0.47017543859649125
--------------------------------------------------------
Report: 
              precision    recall  f1-score   support

          -1       0.44      0.45      0.44       128
           0       0.00      0.00      0.00         9
           1       0.51      0.51      0.51       148

    accuracy                           0.47       285
   macro avg       0.32      0.32      0.32       285
weighted avg       0.46      0.47      0.47       285

Confusion Matrix: 


,Predicted -1,Predicted 0,Predicted 1
Actual -1,58,2,68
Actual 0,5,0,4
Actual 1,70,2,76


In [9]:
x = df2[['sentiment','Volume']]
y = df2['Label']
x_train,x_test,y_train,y_test = train_test_split(x,y,train_size=0.8,random_state=123)
y_train = y_train.values.reshape(-1,1)
y_test = y_test.values.reshape(-1,1)

clf = RandomForestClassifier(n_estimators = 100)
clf.fit(x_train,y_train.ravel())
y_pred = clf.predict(x_test)
print("Accuracy Score: ")
print(accuracy_score(y_test,y_pred))
print("--------------------------------------------------------")
print("Report: ")
print(classification_report(y_test,y_pred))

cm = confusion_matrix(y_test,y_pred)
cm_df = pd.DataFrame(cm, index=["Actual -1", "Actual 0","Actual 1"],columns=["Predicted -1", "Predicted 0","Predicted 1"])
print("Confusion Matrix: ")
display(cm_df)

Accuracy Score: 
0.5298245614035088
--------------------------------------------------------
Report: 
              precision    recall  f1-score   support

          -1       0.49      0.52      0.50       128
           0       0.00      0.00      0.00         9
           1       0.58      0.57      0.58       148

    accuracy                           0.53       285
   macro avg       0.36      0.36      0.36       285
weighted avg       0.52      0.53      0.52       285

Confusion Matrix: 


,Predicted -1,Predicted 0,Predicted 1
Actual -1,66,2,60
Actual 0,7,0,2
Actual 1,62,1,85


In [10]:
x = df2[['sentiment','polarity']]
y = df2['Label']
x_train,x_test,y_train,y_test = train_test_split(x,y,train_size=0.8,random_state=123)
y_train = y_train.values.reshape(-1,1)
y_test = y_test.values.reshape(-1,1)

clf = RandomForestClassifier(n_estimators = 100)
clf.fit(x_train,y_train.ravel())
y_pred = clf.predict(x_test)
print("Accuracy Score: ")
print(accuracy_score(y_test,y_pred))
print("--------------------------------------------------------")
print("Report: ")
print(classification_report(y_test,y_pred))

cm = confusion_matrix(y_test,y_pred)
cm_df = pd.DataFrame(cm, index=["Actual -1", "Actual 0","Actual 1"],columns=["Predicted -1", "Predicted 0","Predicted 1"])
print("Confusion Matrix: ")
display(cm_df)

Accuracy Score: 
0.5157894736842106
--------------------------------------------------------
Report: 
              precision    recall  f1-score   support

          -1       0.48      0.66      0.56       128
           0       0.00      0.00      0.00         9
           1       0.58      0.42      0.49       148

    accuracy                           0.52       285
   macro avg       0.35      0.36      0.35       285
weighted avg       0.52      0.52      0.50       285

Confusion Matrix: 


/Users/taoyu/opt/anaconda3/envs/anly-580/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/taoyu/opt/anaconda3/envs/anly-580/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/taoyu/opt/anaconda3/envs/anly-580/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier

,Predicted -1,Predicted 0,Predicted 1
Actual -1,85,0,43
Actual 0,7,0,2
Actual 1,86,0,62


In [11]:
df3 = pd.read_csv("../Data/Combined_Delayed_2Days.csv")
df3 = df3.replace({'positive':1,'negative':-1,'neutral':0,'Increase':1,'Decrease':-1,'No Change':0})
df3.head()

,Date,Tweets_clean_more,polarity,subjectivity,sentiment,Label_delayed_2Days
0,2013-01-10,to super clear i dont wish could i mandat anyt...,0.216667,0.525000,1,-1
1,2019-01-10,ye get regulatori pushback may avail region go...,0.123333,0.490556,1,1
2,2020-01-10,dome barrel weld made bar pretti good bar need...,0.411111,0.570988,1,-1
3,2012-01-11,the exec conf room tesla use call denali i dec...,0.600000,1.000000,1,-1
4,2016-01-11,sad hear david bowi die he amaz,-0.500000,1.000000,-1,1


In [21]:
y_3 = df3['Label_delayed_2Days']
x_3 = df3[['polarity','sentiment']]
x_train_3,x_test_3,y_train_3,y_test_3 = train_test_split(x_3,y_3,train_size=0.8,random_state=123)
y_train_3 = y_train_3.values.reshape(-1,1)
y_test_3 = y_test_3.values.reshape(-1,1)

In [24]:
clf3 = RandomForestClassifier(n_estimators = 100)
clf3.fit(x_train_3,y_train_3)
y_pred_3 = clf3.predict(x_test_3)
print("Accuracy Score: ")
print(accuracy_score(y_test_3,y_pred_3))
print("--------------------------------------------------------")
print("Report: ")
print(classification_report(y_test_3,y_pred_3)) 

/var/folders/cm/0wzcg0w93xz1qzldlyzp1w2m0000gn/T/ipykernel_24194/685815381.py:2: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  clf3.fit(x_train_3,y_train_3)


Accuracy Score: 
0.5140845070422535
--------------------------------------------------------
Report: 
              precision    recall  f1-score   support

          -1       0.52      0.50      0.51       140
           0       0.20      0.20      0.20         5
           1       0.52      0.54      0.53       139

    accuracy                           0.51       284
   macro avg       0.41      0.41      0.41       284
weighted avg       0.51      0.51      0.51       284

